In [1]:
import logging
from context import pyTTRPGsimulator as rpg
rpg.setup_logging(logging.INFO)


%reload_ext autoreload
%autoreload 2

This is a test error message from module actors


In [2]:
attributes = rpg.actor_attributes

print(attributes)
print(attributes + attributes)
print(attributes == attributes)

Attributes(physical_defense=8, mystical_defense=8, physical_damage_reduction=0, mystical_damage_reduction=0, max_health_points=10, max_stamina_points=0, max_grit_points=0, max_mana_points=0, might=0, agility=0, intelligence=0, charisma=0, prime_modifier_bonus=0, might_save=0, agility_save=0, intelligence_save=0, charisma_save=0, might_check=0, agility_check=0, intelligence_check=0, charisma_check=0, initiative=0, move_speed=5, might_adv=0, agility_adv=0, intelligence_adv=0, charisma_adv=0, combat_mastery=0, spell_DC=0, max_action_points=4, max_attack_before_penalty=1, critical_hit_damage=2, critical_hit_threshold=20, heavy_hit_damage=1, brutal_hit_damage=1, death_threshold=0, mastery_light_armor=True, mastery_heavy_armor=False, is_bleeding=False, is_blinded=False, is_burning=False, is_charmed=False, is_dazed=False, is_deafened=False, is_doomed=False, is_exposed=False, is_frightened=False, is_grappled=False, is_hindered=False, is_impaired=False, is_incapacitated=False, is_intimidated=Fa

In [3]:
# Creating some damage modifiers
fire_vulnerability = rpg.Trait(name="fire_vulnerability",
                               damage_modifiers=rpg.Vulnerability(damage_type=rpg.Fire(), 
                                                                  value=2, is_multiplicative=True))
fire_resistance = rpg.Trait(name="fire_resistance",
                            damage_modifiers=rpg.Resistance(damage_type=rpg.Fire(), value=1))
fire_immune = rpg.Trait(name="fire_immune",
                        damage_modifiers=rpg.Resistance(damage_type=rpg.Fire(), 
                                                        value=0, is_multiplicative=True))

# Creating some damage
fire_damage = rpg.Damage(damage_type=rpg.Fire(), value=4)
physical_damage = rpg.Damage(damage_type=rpg.Slashing(), value=2)

In [6]:
# Creating some items
axe = rpg.MeleeWeapon(name="Regular Axe", damages=[physical_damage])
sword = rpg.MeleeWeapon(name="Flaming Sword", damages=[physical_damage, fire_damage])  # Current limitation : only first damage gets any bonus
shield = rpg.Shield(name="Targe", physical_defense=2, physical_damage_reduction=1)
light_armor = rpg.Armor(physical_defense=4, traits=[fire_immune], name="Novice Armor")
heavy_armor = rpg.Armor(physical_defense=6, traits=[fire_resistance], name="Plate Armor")


# Adding damage modifiers to items
sword.add_trait([fire_vulnerability])
sword.weapon_styles = [rpg.Sword_style()]
shield.add_trait([fire_resistance])

here Regular Axe [Damage(damage_type=Slashing, value=2)] 1 None None
here Flaming Sword [Damage(damage_type=Slashing, value=2), Damage(damage_type=Fire, value=4)] 1 None None


In [7]:
print(sword.damage_modifiers)

[Vulnerability(Damage Type=Fire, value=2, is_multiplicative=True)]


In [8]:
# Creating an actor  (stats are 0 by default)
hero_1 = rpg.Actor(max_health_points=20, combat_mastery=3, might=3, max_mana_points=5, name="hero_1")
hero_2 = rpg.Actor(max_health_points=20, combat_mastery=3, might=3, name="hero_2")

# Some status effects : the name is required for now to differentiate when a similar effect is added twice
bleeding = rpg.Trait(duration=4, is_bleeding=True, name="Small bleeding")
blessed = rpg.Trait(duration=4, has_magic_weapon=True, name="Benediction")


# Attributes are dynamically added bia setters to the Actor class :
print(hero_1.attributes.might)
print(hero_1.might)

print(hero_1.current_action_points)
print(hero_1.current_health_points)
print(hero_1.combat_mastery)

3
3
4
20
3


In [9]:
print(hero_1.attributes.physical_defense, hero_1.attributes.physical_damage_reduction)
hero_1.add_item([shield, light_armor, sword])  # items are unique, and cannot be added twice to an actor
print(hero_1.attributes.physical_defense, hero_1.attributes.physical_damage_reduction)

print("shield", shield.damage_modifiers)
print("light_armor", light_armor.damage_modifiers)
print("sword", sword.damage_modifiers)

8 0
14 1
shield [Resistance(Damage Type=Fire, value=1, is_multiplicative=False)]
light_armor [Resistance(Damage Type=Fire, value=0, is_multiplicative=True)]
sword [Vulnerability(Damage Type=Fire, value=2, is_multiplicative=True)]


In [10]:
print(hero_1.traits_manager)
hero_1.add_trait(bleeding)
print(hero_1.traits_manager)
hero_1.new_round()
print(hero_1.traits_manager)

traits: 
traits: Small bleeding (Duration: 4)
traits: Small bleeding (Duration: 3)


In [ ]:
# Display resistances and vulnerabilities
print("Sword", sword)
print("Weapons:", hero_1.weapons)
print("Resistances:", hero_1.resistances)
print("Vulnerabilities:", hero_1.vulnerabilities)
print("Modifiers:", hero_1.damage_modifiers)

In [ ]:
# Simulate taking damage
damages = [fire_damage, physical_damage]
hero_1.take_damage(damages)

print("total_physical_defense:", hero_1.physical_defense)
print("total_physical_defense:", hero_1.mystical_defense)
print("health_points:", hero_1.current_health_points)
print("modifiers:", hero_1.damage_modifiers)

In [ ]:
#   Try attacks
hero_1.full_rest()
hero_2.full_rest()


attack = rpg.Attack()
attack.execute(hero_1, [hero_2])  # we can wrap this nicely

print(hero_1.advantage_count)
gain_advantage = rpg.GainAdvantage()
gain_advantage.execute(hero_1, [hero_1])
print(hero_1.advantage_count)

In [ ]:
# Create a Fireball spell

damage_action_success = rpg.InflictDamage(
    damages=[rpg.Damage(damage_type=rpg.Fire(), value=10)])

damage_action_failure = rpg.InflictDamage(
    damages=[rpg.Damage(damage_type=rpg.Fire(), value=20)])

fire_condition = rpg.StatusEffect(duration=5, is_burning=True, name="Burning")

fireball_action = rpg.ImposeSavingThrow(
    stat='Agility',
    difficulty=15,
    on_success=[damage_action_success],
    on_failure=[damage_action_failure, fire_condition]
)

fireball = rpg.Spell(actions=[fireball_action], school="Elemental", name="Fireball")

fireball.execute(hero_1, [hero_2])

In [ ]:
fireball.execute(hero_1, [hero_2])

print("modifiers:", hero_2.modifiers)


## Test combat initiative

In [ ]:
# Example usage:
players = [rpg.Actor(name="Player 1", health_points=21, Might=4, equipped_items=[sword]),
           rpg.Actor(name="Player 2", health_points=22, Might=4, equipped_items=[sword])]
monsters = [rpg.Actor(name="Monster 1", health_points=21, Might=4, equipped_items=[sword]),
            rpg.Actor(name="Monster 2", health_points=22, Might=4, equipped_items=[sword])]
combat_manager = rpg.CombatManager(players, monsters, dc=1)

combat_manager.roll_initiative()


# Test combat

In [ ]:
combat_manager.reset_combat()
num_rounds, num_turns, winning_team, remaining_hp = combat_manager.run_combat()
print("num_rounds = ", num_rounds)
print("num_turns = ", num_turns)
print("winning_team = ", winning_team)
print(combat_manager.fight_debrief())


# Statistical results

In [ ]:
players = [rpg.Actor(name="Hero 1", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                      target_mode="target_weakest", strategy=rpg.HelpAllyStrategy()),
           rpg.Actor(name="Hero 2", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                      target_mode="target_weakest", strategy=rpg.HelpAllyStrategy())]

monsters = [rpg.Actor(name="Wolf 1", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                       target_mode="target_weakest", strategy=rpg.DefaultStrategy()),
            rpg.Actor(name="Wolf 2", health_points=15, Might=4, physical_defense=15, equipped_items=[sword], 
                       target_mode="target_weakest", strategy=rpg.DefaultStrategy())]

num_rounds_list, num_turns_list, winrate_A, remaining_hp_list = rpg.run_simulations(5000, players, monsters, dc=15)
print(winrate_A)

In [ ]:
rpg.plot_simulation_results(num_turns_list, winrate_A, remaining_hp_list)